# Deploy Firemind's FCA Compliance Model 8B Instruct Model Package from AWS Marketplace 




AI-powered compliance assistant trained on the complete FCA Handbook. Provides expert guidance on UK financial regulations, compliance requirements, and regulatory changes for financial services firms seeking or maintaining FCA authorization.

This sample notebook shows you how to deploy [Firemind's FCA Compliance Model](https://aws.amazon.com/marketplace/pp/prodview-x0idzsg6qgb0fctuzhm5qwe7) using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [Firemind's FCA Compliance Model](https://aws.amazon.com/marketplace/pp/prodview-x0idzsg6qgb0fctuzhm5qwe7). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Firemind's FCA Compliance Model](https://aws.amazon.com/marketplace/pp/prodview-x0idzsg6qgb0fctuzhm5qwe7)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
# === CONFIGURATION SECTION ===
# Replace these values with your actual AWS resources

# 1. Model Package ARN from AWS Marketplace subscription
# Get this from: https://aws.amazon.com/marketplace/pp/prodview-x0idzsg6qgb0fctuzhm5qwe7
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

# 2. SageMaker Execution Role ARN
# Create or use existing role with SageMaker permissions
role_arn = "arn:aws:iam::YOUR_ACCOUNT_ID:role/SageMakerExecutionRole"

# 3. Model Artifact S3 URI (will be provided by the model package)
model_data_s3_uri = "s3://YOUR_BUCKET/path/to/model.tar.gz"

# 4. Container Image URI (for hosting the model)
container_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.2.0-gpu-py39-cu118-ubuntu20.04"

# 5. Deployment Configuration
model_name = "fca-compliance-model"
endpoint_config_name = f"{model_name}-cfg"
endpoint_name = f"{model_name}-ep"
instance_type = "ml.g5.2xlarge"  # Adjust based on your needs and budget
instance_count = 1

print("Configuration loaded. Please update the placeholder values above before proceeding.")

In [ ]:
import os
import json
import time
import boto3
from botocore.exceptions import ClientError
from datetime import datetime
import io
print('boto3 version:', boto3.__version__)

In [ ]:
# === AWS CONFIGURATION ===
profile_name = os.environ.get('AWS_PROFILE', 'default')
region_name = os.environ.get('AWS_REGION', 'us-east-1')

# Create a Boto3 session using the specified profile
boto3_session = boto3.Session(profile_name=profile_name, region_name=region_name)

# SageMaker runtime client (for invoking endpoints)
sagemaker_runtime = boto3_session.client('sagemaker-runtime', region_name=region_name)

# SageMaker management client (for creating models/endpoints)
sagemaker = boto3_session.client('sagemaker', region_name=region_name)

# Marketplace client for model package operations
marketplace = boto3_session.client('marketplace-catalog', region_name=region_name)

print('Using profile:', profile_name, 'region:', region_name)

# === CONFIGURATION VALIDATION ===
def validate_configuration():
    """Validate that all required configuration values are set."""
    issues = []
    
    if model_package_arn == "<Customer to specify Model package ARN corresponding to their AWS region>":
        issues.append("❌ model_package_arn is not set")
    
    if role_arn == "arn:aws:iam::YOUR_ACCOUNT_ID:role/SageMakerExecutionRole":
        issues.append("❌ role_arn is not set")
    
    if model_data_s3_uri == "s3://YOUR_BUCKET/path/to/model.tar.gz":
        issues.append("❌ model_data_s3_uri is not set")
    
    if container_image == "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.2.0-gpu-py39-cu118-ubuntu20.04":
        print("⚠️  Using default container image. Verify this is correct for your model.")
    
    if issues:
        print("🚨 Configuration Issues Found:")
        for issue in issues:
            print(f"  {issue}")
        print("\nPlease update the configuration values before proceeding.")
        return False
    else:
        print("✅ Configuration validation passed!")
        return True

# === SECURITY & PERMISSION VALIDATION ===
def validate_aws_permissions():
    """Validate that the required AWS permissions are available."""
    required_permissions = [
        'sagemaker:CreateModel',
        'sagemaker:CreateEndpoint',
        'sagemaker:CreateEndpointConfig',
        'sagemaker:InvokeEndpoint',
        'sagemaker:DescribeModel',
        'sagemaker:DescribeEndpoint',
        'sagemaker:DescribeEndpointConfig',
        'sagemaker:CreateTransformJob',
        'sagemaker:DescribeTransformJob',
        'iam:PassRole'
    ]
    
    print("🔐 Validating AWS permissions...")
    
    # Test basic SageMaker access
    try:
        sagemaker.list_models(MaxResults=1)
        print("✅ SageMaker access confirmed")
    except ClientError as e:
        print(f"❌ SageMaker access failed: {e}")
        return False
    
    # Test IAM role access
    try:
        iam = boto3_session.client('iam')
        iam.get_role(RoleName=role_arn.split('/')[-1])
        print("✅ IAM role access confirmed")
    except ClientError as e:
        print(f"❌ IAM role access failed: {e}")
        return False
    
    print("✅ Permission validation completed")
    return True

def validate_model_package_access():
    """Validate access to the model package."""
    if model_package_arn == "<Customer to specify Model package ARN corresponding to their AWS region>":
        print("⚠️  Model package ARN not configured")
        return False
    
    try:
        model_package = get_model_package_details(model_package_arn)
        print(f"✅ Model package access confirmed: {model_package['ModelPackageName']}")
        return True
    except ClientError as e:
        print(f"❌ Model package access failed: {e}")
        return False

# Run all validations
print("🔍 Running comprehensive validation...")
config_valid = validate_configuration()
perms_valid = validate_aws_permissions()
package_valid = validate_model_package_access()

if config_valid and perms_valid and package_valid:
    print("\n✅ All validations passed! Ready to proceed with deployment.")
else:
    print("\n❌ Some validations failed. Please address the issues above before proceeding.")


## 2. Create an endpoint and perform real-time inference

In [ ]:
# === MODEL PACKAGE INTEGRATION ===
def get_model_package_details(model_package_arn):
    """Get model package details from SageMaker."""
    try:
        response = sagemaker.describe_model_package(ModelPackageName=model_package_arn)
        return response
    except ClientError as e:
        print(f"Error getting model package details: {e}")
        raise

def create_model_from_package(model_package_arn, model_name, role_arn):
    """Create a SageMaker model from a model package."""
    try:
        # Get model package details
        model_package = get_model_package_details(model_package_arn)
        
        # Create model from package
        response = sagemaker.create_model(
            ModelName=model_name,
            PrimaryContainer={
                'Image': model_package['InferenceSpecification']['Containers'][0]['Image'],
                'ModelDataUrl': model_package['InferenceSpecification']['Containers'][0]['ModelDataUrl'],
                'Environment': model_package['InferenceSpecification']['Containers'][0].get('Environment', {})
            },
            ExecutionRoleArn=role_arn
        )
        print(f"✅ Model {model_name} created successfully from package")
        return response
    except ClientError as e:
        print(f"Error creating model from package: {e}")
        raise

# === COST ESTIMATION ===
def estimate_endpoint_cost(instance_type, instance_count, hours=1):
    """Estimate cost for running endpoint (approximate)."""
    # Approximate hourly costs for common instance types
    costs = {
        'ml.g5.2xlarge': 1.21,
        'ml.g5.4xlarge': 2.42,
        'ml.g5.8xlarge': 4.84,
        'ml.g5.12xlarge': 7.26,
        'ml.g5.16xlarge': 9.68,
        'ml.g5.24xlarge': 14.52,
        'ml.g5.48xlarge': 29.04
    }
    
    hourly_cost = costs.get(instance_type, 0) * instance_count
    total_cost = hourly_cost * hours
    
    print(f"💰 Estimated cost for {instance_type} x{instance_count} for {hours} hour(s): ${total_cost:.2f}")
    return total_cost


This section provides utility functions and example usage to:
- register a SageMaker model (CreateModel)
- create an EndpointConfig (CreateEndpointConfig)
- create an Endpoint (CreateEndpoint)

You need the following pieces of information to deploy:
- role_arn: IAM execution role for SageMaker (must allow SageMaker to pull from S3 and create network interfaces)
- model_data_s3_uri: S3 URI of the model artifact (tar.gz) containing model weights/config
- container_image: the container image URI to use for hosting (ECR image that knows how to serve your model). For TGI/DJC models hosted via SageMaker, use the appropriate container image.
- instance_type and instance_count for the endpoint

The functions below are defensive (check for existing resources and optionally update).

In [ ]:
def exists_model(model_name):
    try:
        sagemaker.describe_model(ModelName=model_name)
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == 'ValidationException':
            return False
        raise

def exists_endpoint_config(cfg_name):
    try:
        sagemaker.describe_endpoint_config(EndpointConfigName=cfg_name)
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == 'ValidationException':
            return False
        raise

def exists_endpoint(endpoint_name):
    try:
        sagemaker.describe_endpoint(EndpointName=endpoint_name)
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == 'ValidationException':
            return False
        raise

def wait_for_endpoint(endpoint_name, poll_interval=10, timeout_minutes=30):
    """Poll endpoint status until InService or Failed. Returns final status dict."""
    start = datetime.utcnow()
    timeout = timeout_minutes * 60
    while True:
        resp = sagemaker.describe_endpoint(EndpointName=endpoint_name)
        status = resp['EndpointStatus']
        print(f'[{datetime.utcnow().isoformat()}] Endpoint {endpoint_name} status: {status}')
        if status in ('InService', 'Failed'):
            return resp
        if (datetime.utcnow() - start).total_seconds() > timeout:
            raise TimeoutError(f'Endpoint {endpoint_name} did not become InService within timeout')
        time.sleep(poll_interval)

### A. Create an endpoint

In [ ]:
def create_and_deploy_model_endpoint(
    model_name,
    model_data_s3_uri,
    container_image,
    role_arn,
    endpoint_config_name=None,
    endpoint_name=None,
    instance_type='ml.g5.2xlarge',
    instance_count=1,
    wait=True,
    update_if_exists=False
):
    """Create a SageMaker model, endpoint config and endpoint.
    If update_if_exists==True and endpoint exists, this will create a new endpoint config and call UpdateEndpoint.
    Returns the final endpoint description.
    """
    endpoint_config_name = endpoint_config_name or f'{model_name}-cfg-{int(time.time())}'
    endpoint_name = endpoint_name or f'{model_name}-ep-{int(time.time())}'

    # 1) Create Model
    container_def = {
        'Image': container_image,
        'ModelDataUrl': model_data_s3_uri,
        # Optionally pass env vars to container
        'Environment': {}
    }

    try:
        if exists_model(model_name):
            if update_if_exists:
                print(f'Model {model_name} already exists; will re-register model by deleting and recreating.')
                # Deleting model is optional and must be done carefully if endpoints are still using it.
                try:
                    sagemaker.delete_model(ModelName=model_name)
                except Exception as e:
                    print('Warning: could not delete existing model:', e)
            else:
                print(f'Model {model_name} already exists. Skipping CreateModel.')
        print('Creating model:', model_name)
        sagemaker.create_model(
            ModelName=model_name,
            PrimaryContainer=container_def,
            ExecutionRoleArn=role_arn
        )
    except ClientError as e:
        raise

    # 2) Create Endpoint Config
    production_variants = [
        {
            'VariantName': 'AllTraffic',
            'ModelName': model_name,
            'InitialInstanceCount': instance_count,
            'InstanceType': instance_type,
            'InitialVariantWeight': 1.0
        }
    ]
    try:
        if exists_endpoint_config(endpoint_config_name):
            if update_if_exists:
                print(f'EndpointConfig {endpoint_config_name} already exists; deleting and recreating.')
                sagemaker.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
            else:
                print(f'EndpointConfig {endpoint_config_name} already exists. Skipping CreateEndpointConfig.')
        print('Creating endpoint config:', endpoint_config_name)
        sagemaker.create_endpoint_config(
            EndpointConfigName=endpoint_config_name,
            ProductionVariants=production_variants
        )
    except ClientError as e:
        raise

    # 3) Create or Update Endpoint
    try:
        if exists_endpoint(endpoint_name):
            if update_if_exists:
                print(f'Endpoint {endpoint_name} already exists; updating to new config {endpoint_config_name}')
                sagemaker.update_endpoint(EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name)
            else:
                print(f'Endpoint {endpoint_name} already exists. Skipping CreateEndpoint.')
                
        else:
            print('Creating endpoint:', endpoint_name)
            sagemaker.create_endpoint(EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name)

        if wait:
            desc = wait_for_endpoint(endpoint_name)
            return desc
        else:
            return sagemaker.describe_endpoint(EndpointName=endpoint_name)
    except ClientError as e:
        raise


Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [ ]:
# === DEPLOY MODEL FROM PACKAGE ===
# This section creates and deploys the model using the model package

# Validate configuration before proceeding
if not validate_configuration():
    print("❌ Please fix configuration issues before proceeding.")
else:
    # Show cost estimation
    estimate_endpoint_cost(instance_type, instance_count, hours=1)
    
    print(f"\n🚀 Starting deployment of {model_name}...")
    
    try:
        # Option 1: Use model package (recommended)
        if model_package_arn != "<Customer to specify Model package ARN corresponding to their AWS region>":
            print("📦 Creating model from package...")
            create_model_from_package(model_package_arn, model_name, role_arn)
            
            # Get model package details for container info
            model_package = get_model_package_details(model_package_arn)
            container_image = model_package['InferenceSpecification']['Containers'][0]['Image']
            model_data_s3_uri = model_package['InferenceSpecification']['Containers'][0]['ModelDataUrl']
            
            print(f"✅ Using container: {container_image}")
            print(f"✅ Using model data: {model_data_s3_uri}")
        
        # Create endpoint configuration and deploy
        desc = create_and_deploy_model_endpoint(
            model_name=model_name,
            model_data_s3_uri=model_data_s3_uri,
            container_image=container_image,
            role_arn=role_arn,
            endpoint_config_name=endpoint_config_name,
            endpoint_name=endpoint_name,
            instance_type=instance_type,
            instance_count=instance_count,
            wait=True,
            update_if_exists=True
        )
        
        print('✅ Endpoint deployment completed!')
        print('Final endpoint description:')
        print(json.dumps(desc, indent=2, default=str))
        
    except Exception as e:
        print(f"❌ Deployment failed: {e}")
        print("Please check your configuration and try again.")


### C. Perform real-time inference

Once the endpoint is InService, you can invoke it through the runtime client. Adjust the payload format to what the hosting container expects. Examples earlier in this notebook show both the "inputs" style and the "messages" style used by different model server wrappers.

In [ ]:
def invoke_text_endpoint(endpoint_name, prompt, timeout_seconds=120, max_retries=3):
    """Enhanced endpoint invocation with retry logic and better error handling."""
    payload = {
        'inputs': prompt,
        'parameters': {
            'do_sample': True,
            'max_new_tokens': 256,
            'temperature': 0.2
        }
    }
    
    for attempt in range(max_retries):
        try:
            response = sagemaker_runtime.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType='application/json',
                Body=json.dumps(payload)
            )
            out = response['Body'].read().decode('utf-8')
            return json.loads(out)
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == 'ThrottlingException' and attempt < max_retries - 1:
                wait_time = 2 ** attempt  # Exponential backoff
                print(f"⏳ Throttling detected, retrying in {wait_time} seconds... (attempt {attempt + 1}/{max_retries})")
                time.sleep(wait_time)
                continue
            else:
                print(f'❌ Invoke failed after {attempt + 1} attempts: {e}')
                raise
        except Exception as e:
            print(f'❌ Unexpected error: {e}')
            raise

def safe_invoke_endpoint(endpoint_name, prompt, **kwargs):
    """Safely invoke endpoint with comprehensive error handling."""
    try:
        # Check if endpoint exists and is in service
        if not exists_endpoint(endpoint_name):
            raise ValueError(f"Endpoint {endpoint_name} does not exist")
        
        endpoint_status = sagemaker.describe_endpoint(EndpointName=endpoint_name)
        if endpoint_status['EndpointStatus'] != 'InService':
            raise ValueError(f"Endpoint {endpoint_name} is not in service. Status: {endpoint_status['EndpointStatus']}")
        
        # Invoke endpoint
        result = invoke_text_endpoint(endpoint_name, prompt, **kwargs)
        return result
        
    except Exception as e:
        print(f"❌ Error invoking endpoint: {e}")
        return None

# === EXAMPLE USAGE ===
def test_endpoint_with_sample_queries():
    """Test the endpoint with sample FCA compliance queries."""
    sample_queries = [
        "What are the key FCA compliance requirements for financial services firms?",
        "Explain the regulatory framework for UK banks under FCA supervision.",
        "What are the capital adequacy requirements for FCA-authorized firms?"
    ]
    
    if exists_endpoint(endpoint_name):
        print(f"🧪 Testing endpoint {endpoint_name} with sample queries...")
        
        for i, query in enumerate(sample_queries, 1):
            print(f"\n📝 Query {i}: {query}")
            result = safe_invoke_endpoint(endpoint_name, query)
            
            if result:
                print(f"✅ Response: {result}")
            else:
                print("❌ Failed to get response")
    else:
        print(f"❌ Endpoint {endpoint_name} does not exist or is not ready")

print("🔧 Enhanced endpoint invocation functions loaded.")
print("Use test_endpoint_with_sample_queries() to test your endpoint with sample FCA queries.")


### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
def delete_endpoint_resources(endpoint_name=None, endpoint_config_name=None, model_name=None, wait_for_deletion=True):
    if endpoint_name and exists_endpoint(endpoint_name):
        print('Deleting endpoint:', endpoint_name)
        sagemaker.delete_endpoint(EndpointName=endpoint_name)
        if wait_for_deletion:
            # wait until endpoint no longer exists
            while exists_endpoint(endpoint_name):
                print('Waiting for endpoint deletion...')
                time.sleep(5)
    if endpoint_config_name and exists_endpoint_config(endpoint_config_name):
        print('Deleting endpoint config:', endpoint_config_name)
        sagemaker.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
    if model_name and exists_model(model_name):
        print('Deleting model:', model_name)
        try:
            sagemaker.delete_model(ModelName=model_name)
        except Exception as e:
            print('Could not delete model:', e)


## 3. Perform batch inference


In [ ]:
# === BATCH INFERENCE IMPLEMENTATION ===
import pandas as pd
from sagemaker.s3 import S3Uploader, S3Downloader

def create_batch_transform_job(
    model_name,
    input_s3_uri,
    output_s3_uri,
    instance_type='ml.g5.2xlarge',
    instance_count=1,
    max_payload_size=6,  # MB
    job_name=None
):
    """Create a SageMaker batch transform job."""
    job_name = job_name or f"{model_name}-batch-{int(time.time())}"
    
    try:
        response = sagemaker.create_transform_job(
            TransformJobName=job_name,
            ModelName=model_name,
            MaxPayloadInMB=max_payload_size,
            BatchStrategy='MultiRecord',
            TransformInput={
                'DataSource': {
                    'S3DataSource': {
                        'S3DataType': 'S3Prefix',
                        'S3Uri': input_s3_uri
                    }
                },
                'ContentType': 'application/json',
                'SplitType': 'Line'
            },
            TransformOutput={
                'S3OutputPath': output_s3_uri,
                'Accept': 'application/json'
            },
            TransformResources={
                'InstanceType': instance_type,
                'InstanceCount': instance_count
            }
        )
        print(f"✅ Batch transform job '{job_name}' created successfully")
        return response
    except ClientError as e:
        print(f"❌ Error creating batch transform job: {e}")
        raise

def wait_for_batch_job_completion(job_name, poll_interval=30):
    """Wait for batch transform job to complete."""
    print(f"⏳ Waiting for batch job '{job_name}' to complete...")
    
    while True:
        try:
            response = sagemaker.describe_transform_job(TransformJobName=job_name)
            status = response['TransformJobStatus']
            print(f"📊 Job status: {status}")
            
            if status in ['Completed', 'Failed', 'Stopped']:
                return response
                
            time.sleep(poll_interval)
        except ClientError as e:
            print(f"❌ Error checking job status: {e}")
            raise

def prepare_batch_input(input_data, s3_bucket, s3_prefix):
    """Prepare input data for batch inference."""
    # Convert input data to JSONL format
    if isinstance(input_data, list):
        jsonl_data = '\n'.join([json.dumps(item) for item in input_data])
    else:
        jsonl_data = input_data
    
    # Upload to S3
    s3_uri = f"s3://{s3_bucket}/{s3_prefix}/input.jsonl"
    S3Uploader.upload_string(jsonl_data, s3_uri)
    print(f"✅ Input data uploaded to: {s3_uri}")
    return s3_uri

def process_batch_output(output_s3_uri):
    """Process and download batch inference results."""
    try:
        # Download results
        results = S3Downloader.download(output_s3_uri, './batch_output/')
        print(f"✅ Batch results downloaded to: ./batch_output/")
        
        # Process results
        output_files = [f for f in os.listdir('./batch_output/') if f.endswith('.jsonl')]
        all_results = []
        
        for file in output_files:
            with open(f'./batch_output/{file}', 'r') as f:
                for line in f:
                    if line.strip():
                        all_results.append(json.loads(line.strip()))
        
        return all_results
    except Exception as e:
        print(f"❌ Error processing batch output: {e}")
        raise

# === EXAMPLE BATCH INFERENCE ===
def run_batch_inference_example():
    """Example of running batch inference."""
    # Sample input data
    sample_inputs = [
        {"inputs": "What are the key FCA compliance requirements for financial services?"},
        {"inputs": "Explain the regulatory framework for UK banks."},
        {"inputs": "What are the capital adequacy requirements under FCA rules?"}
    ]
    
    # Configuration
    s3_bucket = "your-batch-inference-bucket"  # Replace with your bucket
    s3_prefix = "fca-batch-inference"
    input_s3_uri = prepare_batch_input(sample_inputs, s3_bucket, f"{s3_prefix}/input")
    output_s3_uri = f"s3://{s3_bucket}/{s3_prefix}/output"
    
    # Create batch transform job
    job_response = create_batch_transform_job(
        model_name=model_name,
        input_s3_uri=input_s3_uri,
        output_s3_uri=output_s3_uri,
        instance_type=instance_type,
        instance_count=1
    )
    
    # Wait for completion
    final_response = wait_for_batch_job_completion(job_response['TransformJobName'])
    
    if final_response['TransformJobStatus'] == 'Completed':
        print("✅ Batch inference completed successfully!")
        results = process_batch_output(output_s3_uri)
        return results
    else:
        print(f"❌ Batch inference failed: {final_response.get('FailureReason', 'Unknown error')}")
        return None

print("📋 Batch inference functions loaded. Use run_batch_inference_example() to run a sample batch job.")


## 📋 Troubleshooting Guide

### Common Issues and Solutions:

#### 1. **Configuration Issues**
- **Issue**: "Configuration validation failed"
- **Solution**: Update all placeholder values in the configuration section
- **Check**: Ensure `model_package_arn`, `role_arn`, and other values are properly set

#### 2. **Permission Issues**
- **Issue**: "Access denied" or "Insufficient permissions"
- **Solution**: Ensure your IAM role has the required SageMaker permissions
- **Required permissions**: `AmazonSageMakerFullAccess` or equivalent

#### 3. **Model Package Issues**
- **Issue**: "Model package not found" or "Access denied to model package"
- **Solution**: Ensure you have subscribed to the FCA Compliance Model on AWS Marketplace
- **Check**: Verify the model package ARN is correct for your region

#### 4. **Endpoint Deployment Issues**
- **Issue**: Endpoint creation fails or times out
- **Solution**: Check instance type availability in your region
- **Alternative**: Try a different instance type (e.g., `ml.g5.4xlarge` instead of `ml.g5.2xlarge`)

#### 5. **Cost Management**
- **Issue**: Unexpected charges
- **Solution**: Always delete endpoints when not in use
- **Tip**: Use the cleanup functions provided in this notebook

#### 6. **Network Issues**
- **Issue**: Endpoint not accessible or slow responses
- **Solution**: Check VPC configuration and security groups
- **Consider**: Using VPC endpoints for better security

### 📞 Support Resources:
- [AWS SageMaker Documentation](https://docs.aws.amazon.com/sagemaker/)
- [AWS Marketplace Support](https://aws.amazon.com/marketplace/help)
- [FCA Compliance Model Support](https://aws.amazon.com/marketplace/pp/prodview-x0idzsg6qgb0fctuzhm5qwe7)

### 💡 Best Practices:
- Always test with small workloads first
- Monitor costs using AWS Cost Explorer
- Use appropriate instance types for your workload
- Implement proper error handling in production
- Consider using serverless endpoints for variable workloads


## 4. Clean-up

### A. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

